In [1]:
from plotly.offline import init_notebook_mode, iplot
from wordcloud import WordCloud
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import plotly.plotly as py
from plotly import tools
from datetime import date
import pandas as pd
import numpy as np 
import seaborn as sns
import random 
import warnings
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

train_all = pd.read_csv("./data/train.csv", parse_dates=["activation_date"])
test_all = pd.read_csv("./data/test.csv", parse_dates=["activation_date"])
#train_all = train_all[:10000]

In [2]:
train_all.image_top_1.head(100)

0     1008.0
1      692.0
2     3032.0
3      796.0
4     2264.0
5      796.0
6     2823.0
7      567.0
8      415.0
9       46.0
10    1396.0
11     119.0
12    1040.0
13    2849.0
14    2133.0
15     408.0
16    1118.0
17     983.0
18      95.0
19       NaN
20    2011.0
21       NaN
22    2264.0
23      87.0
24     843.0
25     957.0
26    1420.0
27     436.0
28     618.0
29      63.0
       ...  
70      45.0
71     540.0
72      93.0
73     865.0
74       NaN
75     294.0
76      86.0
77      42.0
78    2218.0
79    2638.0
80      87.0
81      52.0
82    3063.0
83    1993.0
84     421.0
85    1121.0
86      71.0
87     787.0
88      95.0
89    2764.0
90    2895.0
91       NaN
92       NaN
93     625.0
94     560.0
95    1437.0
96      88.0
97    3022.0
98    2262.0
99      47.0
Name: image_top_1, Length: 100, dtype: float64

In [2]:
train_all['weekday'] = train_all.activation_date.dt.weekday
train_all['month'] = train_all.activation_date.dt.month
train_all['day'] = train_all.activation_date.dt.day

train_all['description'] = train_all['description'].fillna(" ")
train_all['description_len'] = train_all['description'].apply(lambda x : len(x.split()))

train_all['title'] = train_all['title'].fillna(" ")
train_all['title_len'] = train_all['title'].apply(lambda x : len(x.split()))

In [3]:
test_all['weekday'] = test_all.activation_date.dt.weekday
test_all['month'] = test_all.activation_date.dt.month
test_all['day'] = test_all.activation_date.dt.day

test_all['description'] = test_all['description'].fillna(" ")
test_all['description_len'] = test_all['description'].apply(lambda x : len(x.split()))

test_all['title'] = test_all['title'].fillna(" ")
test_all['title_len'] = test_all['title'].apply(lambda x : len(x.split()))

In [4]:
parent_category_name_map = {"Личные вещи" : "Personal belongings",
                            "Для дома и дачи" : "For the home and garden",
                            "Бытовая электроника" : "Consumer electronics",
                            "Недвижимость" : "Real estate",
                            "Хобби и отдых" : "Hobbies & leisure",
                            "Транспорт" : "Transport",
                            "Услуги" : "Services",
                            "Животные" : "Animals",
                            "Для бизнеса" : "For business"}

region_map = {"Свердловская область" : "Sverdlovsk oblast",
            "Самарская область" : "Samara oblast",
            "Ростовская область" : "Rostov oblast",
            "Татарстан" : "Tatarstan",
            "Волгоградская область" : "Volgograd oblast",
            "Нижегородская область" : "Nizhny Novgorod oblast",
            "Пермский край" : "Perm Krai",
            "Оренбургская область" : "Orenburg oblast",
            "Ханты-Мансийский АО" : "Khanty-Mansi Autonomous Okrug",
            "Тюменская область" : "Tyumen oblast",
            "Башкортостан" : "Bashkortostan",
            "Краснодарский край" : "Krasnodar Krai",
            "Новосибирская область" : "Novosibirsk oblast",
            "Омская область" : "Omsk oblast",
            "Белгородская область" : "Belgorod oblast",
            "Челябинская область" : "Chelyabinsk oblast",
            "Воронежская область" : "Voronezh oblast",
            "Кемеровская область" : "Kemerovo oblast",
            "Саратовская область" : "Saratov oblast",
            "Владимирская область" : "Vladimir oblast",
            "Калининградская область" : "Kaliningrad oblast",
            "Красноярский край" : "Krasnoyarsk Krai",
            "Ярославская область" : "Yaroslavl oblast",
            "Удмуртия" : "Udmurtia",
            "Алтайский край" : "Altai Krai",
            "Иркутская область" : "Irkutsk oblast",
            "Ставропольский край" : "Stavropol Krai",
            "Тульская область" : "Tula oblast"}


category_map = {"Одежда, обувь, аксессуары":"Clothing, shoes, accessories",
"Детская одежда и обувь":"Children's clothing and shoes",
"Товары для детей и игрушки":"Children's products and toys",
"Квартиры":"Apartments",
"Телефоны":"Phones",
"Мебель и интерьер":"Furniture and interior",
"Предложение услуг":"Offer services",
"Автомобили":"Cars",
"Ремонт и строительство":"Repair and construction",
"Бытовая техника":"Appliances",
"Товары для компьютера":"Products for computer",
"Дома, дачи, коттеджи":"Houses, villas, cottages",
"Красота и здоровье":"Health and beauty",
"Аудио и видео":"Audio and video",
"Спорт и отдых":"Sports and recreation",
"Коллекционирование":"Collecting",
"Оборудование для бизнеса":"Equipment for business",
"Земельные участки":"Land",
"Часы и украшения":"Watches and jewelry",
"Книги и журналы":"Books and magazines",
"Собаки":"Dogs",
"Игры, приставки и программы":"Games, consoles and software",
"Другие животные":"Other animals",
"Велосипеды":"Bikes",
"Ноутбуки":"Laptops",
"Кошки":"Cats",
"Грузовики и спецтехника":"Trucks and buses",
"Посуда и товары для кухни":"Tableware and goods for kitchen",
"Растения":"Plants",
"Планшеты и электронные книги":"Tablets and e-books",
"Товары для животных":"Pet products",
"Комнаты":"Room",
"Фототехника":"Photo",
"Коммерческая недвижимость":"Commercial property",
"Гаражи и машиноместа":"Garages and Parking spaces",
"Музыкальные инструменты":"Musical instruments",
"Оргтехника и расходники":"Office equipment and consumables",
"Птицы":"Birds",
"Продукты питания":"Food",
"Мотоциклы и мототехника":"Motorcycles and bikes",
"Настольные компьютеры":"Desktop computers",
"Аквариум":"Aquarium",
"Охота и рыбалка":"Hunting and fishing",
"Билеты и путешествия":"Tickets and travel",
"Водный транспорт":"Water transport",
"Готовый бизнес":"Ready business",
"Недвижимость за рубежом":"Property abroad"}

In [5]:
train_all['region_en'] = train_all['region'].apply(lambda x : region_map[x])
train_all['parent_category_name_en'] = train_all['parent_category_name'].apply(lambda x : parent_category_name_map[x])
train_all['category_name_en'] = train_all['category_name'].apply(lambda x : category_map[x])

In [6]:
test_all['region_en'] = train_all['region'].apply(lambda x : region_map[x])
test_all['parent_category_name_en'] = train_all['parent_category_name'].apply(lambda x : parent_category_name_map[x])
test_all['category_name_en'] = train_all['category_name'].apply(lambda x : category_map[x])

In [7]:
#cols_to_drop = ["item_id", "user_id", "title", "description", "activation_date", "image", "region", "city"]
#train_refactored = train_all.drop(cols_to_drop, axis=1)

In [8]:
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')

txt = " ".join(train_all.title)
import re
words = re.findall(r'\w+', txt)


r = re.compile("[a-zA-Z]+")
english_words = [w for w in filter(r.match, words)]
txt = " ".join(english_words)


words = re.findall(r'\w+', txt)
count_words = {}
for word in words:
    if word in count_words:
        count_words[word] += 1
    else:
        count_words[word] = 1

i = 0
brands_train = []
my_stop_words = ['colin', 'black', 'love', 'new', 'back']
for w in sorted(count_words, key=count_words.get, reverse=True):
    i += 0
    if (len(str(w)) > 2) and str(w).lower() not in brands_train and str(w).lower() not in english_stopwords and str(w).lower() not in my_stop_words:
        brands_train.append(str(w).lower())
#     if i > 1000:
#         break

In [9]:
len(brands_train)

62581

In [10]:
brands_file = open('./data/brans_train.txt', 'w')
for brand in brands_train:
    brands_file.write("%s\n" % brand)

In [11]:
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')

txt = " ".join(test_all.title)
import re
words = re.findall(r'\w+', txt)


r = re.compile("[a-zA-Z]+")
english_words = [w for w in filter(r.match, words)]
txt = " ".join(english_words)


words = re.findall(r'\w+', txt)
count_words = {}
for word in words:
    if word in count_words:
        count_words[word] += 1
    else:
        count_words[word] = 1

i = 0
brands_test = []
my_stop_words = ['colin', 'black', 'love', 'new', 'back']
for w in sorted(count_words, key=count_words.get, reverse=True):
    i += 0
    if (len(str(w)) > 2) and str(w).lower() not in brands_test and str(w).lower() not in english_stopwords and str(w).lower() not in my_stop_words:
        brands_test.append(str(w).lower())
#     if i > 1000:
#         break

In [12]:
len(brands_test)

31347

In [13]:
brands_file = open('./data/brans_test.txt', 'w')
for brand in brands_test:
    brands_file.write("%s\n" % brand)

In [14]:
from tqdm import tqdm
train_all['brand'] = 'None'
for index, row in tqdm(train_all.iterrows()):
    text = (row['title'] + " " + row['description']).lower()
    
    for brand in brands_train:
        if brand in text:
            train_all.at[index, 'brand'] = brand
            break;

1503424it [3:49:01, 109.41it/s]


In [15]:
text_classes = pd.read_csv('./data/text_classes.csv')

In [16]:
train_all['text_class'] = text_classes.text_class.values

In [18]:
train_all.head(50)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,...,weekday,month,day,description_len,title_len,region_en,parent_category_name_en,category_name_en,brand,text_class
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),...,1,3,28,7,3,Sverdlovsk oblast,Personal belongings,Children's products and toys,None,0
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,...,6,3,26,7,3,Samara oblast,For the home and garden,Furniture and interior,None,0
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,...,0,3,20,17,2,Rostov oblast,Consumer electronics,Audio and video,philips,0
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,...,5,3,25,3,1,Tatarstan,Personal belongings,Children's products and toys,None,0
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",...,3,3,16,4,3,Volgograd oblast,Transport,Cars,None,0
5,51e0962387f7,bbfad0b1ad0a,Татарстан,Чистополь,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Авто люлька,...,1,3,28,3,2,Tatarstan,Personal belongings,Children's products and toys,None,0
6,c4f260a2b48a,08f469d2e6f7,Нижегородская область,Нижний Новгород,Для дома и дачи,Ремонт и строительство,Сантехника и сауна,NaN,NaN,Водонагреватель 100 литров нержавейка плоский,...,3,3,23,21,5,Nizhny Novgorod oblast,For the home and garden,Repair and construction,None,0
7,6b71309d6a8a,fef86baa002c,Пермский край,Пермь,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Джинсы,26,Бойфренды colins,...,5,3,25,4,2,Perm Krai,Personal belongings,"Clothing, shoes, accessories",colins,0
8,c5b969cb63a2,055825270190,Оренбургская область,Оренбург,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,> 50 (XXL),Платье,...,4,3,17,5,1,Orenburg oblast,Personal belongings,"Clothing, shoes, accessories",None,0
9,b1570962e68c,f9e8f831d94c,Нижегородская область,Нижний Новгород,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,25,Полу ботиночки замш натур.Бамбини,...,2,3,22,11,4,Nizhny Novgorod oblast,Personal belongings,Children's clothing and shoes,None,0


In [19]:
train_all.to_csv('./data/try_2_08_05_2018.csv')

In [20]:
from tqdm import tqdm
test_all['brand'] = 'None'
for index, row in tqdm(test_all.iterrows()):
    text = (row['title'] + " " + row['description']).lower()
    
    for brand in brands_test:
        if brand in text:
            test_all.at[index, 'brand'] = brand
            break;

508438it [40:47, 207.76it/s]


In [21]:
text_classes_test = pd.read_csv('./data/text_classes_test.csv')

In [22]:
test_all['text_class'] = text_classes_test.text_class.values

In [24]:
test_all.head(50)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,...,weekday,month,day,description_len,title_len,region_en,parent_category_name_en,category_name_en,brand,text_class
0,6544e41a8817,dbe73ad6e4b5,Волгоградская область,Волгоград,Личные вещи,Детская одежда и обувь,Для мальчиков,Обувь,25,Отдам бесплатно,...,1,4,18,2,2,Sverdlovsk oblast,Personal belongings,Children's products and toys,None,0
1,65b9484d670f,2e11806abe57,Свердловская область,Нижняя Тура,Хобби и отдых,Велосипеды,Дорожные,NaN,NaN,Продам велосипед,...,6,4,16,10,2,Samara oblast,For the home and garden,Furniture and interior,f200,0
2,8bab230b2ecd,0b850bbebb10,Новосибирская область,Бердск,Бытовая электроника,Аудио и видео,Телевизоры и проекторы,NaN,NaN,BBK,...,0,4,17,21,1,Rostov oblast,Consumer electronics,Audio and video,bbk,0
3,8e348601fefc,5f1d5c3ce0da,Саратовская область,Саратов,Для дома и дачи,Бытовая техника,Для кухни,Вытяжки,NaN,Вытяжка Jetair 60,...,0,4,17,12,3,Tatarstan,Personal belongings,Children's products and toys,air,0
4,8bd2fe400b89,23e2d97bfc7f,Оренбургская область,Бузулук,Личные вещи,Товары для детей и игрушки,Детские коляски,NaN,NaN,Коляска зима-лето,...,5,4,15,10,2,Volgograd oblast,Transport,Cars,None,0
5,c63dbd6c657f,c2a632af2602,Ростовская область,Ростов-на-Дону,Для дома и дачи,Ремонт и строительство,Окна и балконы,NaN,NaN,Стекла для пластиковых окон,...,2,4,12,28,4,Tatarstan,Personal belongings,Children's products and toys,None,0
6,6d1a410df86e,b239811ad530,Оренбургская область,Оренбург,Бытовая электроника,Ноутбуки,NaN,NaN,NaN,Ноутбук Lenovo Ideapad 300-15Ibr арт.14899,...,0,4,17,220,5,Nizhny Novgorod oblast,For the home and garden,Repair and construction,lenovo,0
7,e8d3e7922b80,d85fa02e6341,Калининградская область,Калининград,Бытовая электроника,Телефоны,Аксессуары,Чехлы и плёнки,NaN,Nillkin Phenom для Apple iPhone,...,1,4,18,111,5,Perm Krai,Personal belongings,"Clothing, shoes, accessories",iphone,0
8,2bc1ab208462,ae6586719bec,Новосибирская область,Новосибирск,Бытовая электроника,Товары для компьютера,Флэшки и карты памяти,NaN,NaN,Флешка USB 32Gb smartbuy,...,1,4,18,34,4,Orenburg oblast,Personal belongings,"Clothing, shoes, accessories",art,0
9,7e05d77a9181,30ad26d633ef,Свердловская область,Полевской,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,23,Ботинки,...,2,4,12,2,1,Nizhny Novgorod oblast,Personal belongings,Children's clothing and shoes,None,0


In [25]:
test_all.to_csv('./data/try_2_29_05_2018_test.csv')